In [4]:
import os
import pickle
from tensorflow.keras.layers import TextVectorization
import tensorflow as tf
import numpy as np
import pandas as pd
import gradio as gr


In [2]:

df=pd.read_csv(os.path.join('data', 'train.csv'))


#defining the Independent variables(Features) X from the depenedent varaiable Y
X=df['comment_text'].values # the independent variable
y=df[df.columns[2:]].values # dependent variables

#init maxima words that would be held as vocabs; many words increase model accuracy whilst slowing training process
MAX_VOCAB=200000 

# init textVectorisation that will be used to map text features into respective integer values
vectorizer=TextVectorization(max_tokens=MAX_VOCAB,output_sequence_length=2000,output_mode='int')

# make the vactorizer learn words from the X features
vectorizer.adapt(X)

text_vectorized=vectorizer('hey man your ugly face is annoying')

model=tf.keras.models.load_model(
    os.path.join('models', 'RNN_Toxic_Comment_tf_model.h5'),
    custom_objects=None, compile=False)

2024-06-02 17:23:45.498407: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 253720368 exceeds 10% of free system memory.


In [7]:

def results_prediction(text_to_predict):
    text_vectorized=vectorizer(text_to_predict)
    predictions=model.predict(np.expand_dims(text_vectorized,0))
    text=''
    predictions.flatten()
    
    for index, col_value in enumerate(df.columns[2:]):
        text+='{} : {}\n\n'.format(col_value.upper(), predictions[0][index]>0.5)
        
    return text    



interface=gr.Interface(
    fn=results_prediction,
    inputs=gr.Textbox(lines=2, placeholder='enter the text for assessment'),
    outputs='text'
)
    

interface.launch(share=True)


Running on local URL:  http://127.0.0.1:7862

Could not create share link. Missing file: /home/shimmian/anaconda3/lib/python3.11/site-packages/gradio/frpc_linux_amd64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.2
3. Move the file to this location: /home/shimmian/anaconda3/lib/python3.11/site-packages/gradio


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
